In [51]:
veridicality_verbs = ['move', 'decline', 'intend', 'attempt', 'start', 'get', 'help',
       'manage', 'seek', 'agree', 'expect', 'happen','claim', 'wish',
       'wait', 'love', 'work', 'say', 'mean', 'prefer', 'hope', 'prove',
       'seem', 'begin', 'use', 'try', 'continue', 'refuse', 'have',
       'need', 'want', 'like', 'appear', 'plan', 'tend', 'come', 'choose',
       'fail', 'decide', 'remain', 'prepare', 'dare', 'pretend',
       'proceed', 'return', 'cease', 'propose', 'deserve', 'exist',
       'serve', 'aim', 'strive', 'threaten', 'learn', 'promise', 'ask',
       'turn', 'struggle', 'forget', 'reply', 'admit', 'confirm',
       'notice', 'imply', 'declare', 'observe', 'predict', 'assert',
       'feel', 'insist', 'reveal', 'announce', 'worry', 'see', 'remember',
       'demonstrate', 'explain', 'hold', 'complain', 'request',
       'speculate', 'add', 'recommend', 'warn', 'understand', 'demand',
       'show', 'find', 'note', 'saw', 'know', 'state', 'argue',
       'indicate', 'report', 'suggest', 'think', 'believe', 'be',
       'recognize', 'realize', 'determine', 'felt', 'conclude', 'assume',
       'ensure', 'require', 'estimate', 'comment', 'advise', 'doubt',
       'give', 'suspect', 'concern', 'convince', 'acknowledge', 'fear',
       'write', 'tell', 'hear', 'contend', 'provide', 'discover',
       'mention']

In [52]:
signature_list= {'+/+': ['realize that', 'know that', 'remember that','find that',
'notice that', 'reveal that', 'acknowledge that', 'admit that', 'learn that', 'observe that', 'see that', 'note that', 'recognize that',
'understand that', 'discover that'],
'+/−': ['manage to' , 'begin to',
'serve to' , 'start to' , 'dare to' ,'use to', 'get to' , 'come to'],
'−/+': ['forget to', 'fail to'],
'◦/+': ['suspect that', 'explain that' , 'mean to', 'predict that'],
'◦/−': ['attempt to'],
'−/◦': ['refuse to', 'decline to', 'remain to'],
'+/◦': ['show that','confirm that', 'demonstrate that','ensure that',' help to', 'tend to'],
'◦/◦': ['try to','hope that','hope to','mention that','like to','continue to','expect that',
'agree that','love to','reply that','conclude that','say that',
'complain that','speculate that','state that','suggest that',
'worry that','mean that','intend to','insist that','imply that',
'indicate that','plan to','promise to','prove to','saw that',
'seem that','tell that','think that','felt that','write that',
'decide to','assume that','believe that','assert that','concern that','estimate that','convince that','decide that','appear that',
'argue that','aim to','cease to','strive to','proceed to',
'choose to','seem to','prove that','provide that','seek to',
'appear to','comment that','contend that','want to','doubt that','feel that','fear that','agree to','announce that','claim that',
'struggle to','hear that','propose to','wish to','say to','turn to','wish that','work to','advise that','move to','claim to','expect to','report that','happen to','propose that','hold that','declare that',
'prefer to','need to','give that','deserve to','threaten to','exist to','be that','prepare to','wait to','pretend to','ask to','return to','request that','demand that','recommend that','require that']}

In [53]:
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag
import os
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

# load the IBM Debater – Claim Stance Dataset
df = pd.read_csv('article_info.csv')

stem_words = []

stemmer = PorterStemmer()

verd_verbs_dict = {key: 0 for key in veridicality_verbs}

signature_count = {key: 0 for key in signature_list}

for word in veridicality_verbs:
    stem_words.append(stemmer.stem(word))


def count_veridicality_verbs(path):
    with open(path, 'r', encoding='utf-8') as file:
        text = file.read()
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    verb_count = 0
    for token, pos in pos_tags:
        if pos.startswith('V') and stemmer.stem(token.lower()) in stem_words:
            verb  = token.lower()
            if verb in verd_verbs_dict.keys():
                verd_verbs_dict[verb] += 1
            verb_count += 1
    return verb_count


def count_signature_verbs(path):
    with open(path, 'r', encoding='utf-8') as file:
        text = file.read()
    for key, values in signature_list.items():
        for value in values:
            if value in text:
                signature_count[key] += 1
    return signature_count




df[['+/+', '+/-', '-/+', '◦/+','◦/−','−/◦','+/◦','◦/◦']] = df['clean_file'].apply(lambda x: pd.Series(count_signature_verbs(x)))

print(df[['+/+', '+/-', '-/+', '◦/+','◦/−','−/◦','+/◦','◦/◦']].describe())


df['veridicality_count'] = df['clean_file'].apply(lambda x: count_veridicality_verbs(x))

print(df[['veridicality_count']].describe())

total_verbs_count = sum(verd_verbs_dict.values())
verd_verbs_percent = {}

for verb, count in verd_verbs_dict.items():
    percent = round((count / total_verbs_count) * 100, 2)
    verd_verbs_percent[verb] = percent

print(verd_verbs_percent)

print(signature_count)

print("Most frequent verdicality verb:",max(verd_verbs_percent, key=verd_verbs_percent.get))
print("Least frequent verdicality verb:",min(verd_verbs_percent, key=verd_verbs_percent.get))

print("Most frequent signature_verb:",max(signature_count , key=signature_count.get))
print("Least frequent signature_verb:",min(signature_count , key=signature_count.get))



               +/+          +/-          -/+          ◦/+          ◦/−  \
count  1056.000000  1056.000000  1056.000000  1056.000000  1056.000000   
mean    110.945076   194.929924    32.607008     8.965909   117.200758   
std      74.958521   123.497594    20.288206     8.164701    68.837823   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%      45.000000    96.750000    18.000000     2.000000    58.750000   
50%     103.000000   173.000000    27.000000     4.000000   116.000000   
75%     176.000000   299.250000    50.000000    18.000000   173.000000   
max     254.000000   427.000000    72.000000    24.000000   246.000000   

               −/◦          +/◦          ◦/◦  
count  1056.000000  1056.000000  1056.000000  
mean     35.872159   279.089962  1844.358902  
std      24.676557   167.928879  1198.030954  
min       0.000000     0.000000     2.000000  
25%      13.750000   144.500000   849.500000  
50%      32.000000   258.000000  1625.500000  
75% 

In [58]:
# load the Webis-Argument-Framing-19 dataset
df = pd.read_csv('Webis-argument-framing.csv')

verd_verbs_dict = {key: 0 for key in veridicality_verbs}

signature_count = {key: 0 for key in signature_list}


def count_veridicality_verbs(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    verb_count = 0
    for token, pos in pos_tags:
        if pos.startswith('V') and stemmer.stem(token.lower()) in stem_words:
            verb  = token.lower()
            if verb in verd_verbs_dict.keys():
                verd_verbs_dict[verb] += 1
            verb_count += 1
    return verb_count


def count_signature_verbs(text):
    for key, values in signature_list.items():
        for value in values:
            if value in text:
                signature_count[key] += 1
    return signature_count




df[['+/+', '+/-', '-/+', '◦/+','◦/−','−/◦','+/◦','◦/◦']] = df['premise'].apply(lambda x: pd.Series(count_signature_verbs(x)))

print(df[['+/+', '+/-', '-/+', '◦/+','◦/−','−/◦','+/◦','◦/◦']].describe())


df[['+/+', '+/-', '-/+', '◦/+','◦/−','−/◦','+/◦','◦/◦']] = df['conclusion'].apply(lambda x: pd.Series(count_signature_verbs(x)))

print(df[['conclusion','+/+', '+/-', '-/+', '◦/+','◦/−','−/◦','+/◦','◦/◦']].describe())


df['verdicality_count_premise'] = df['premise'].apply(count_veridicality_verbs)

df['verdicality_count_conclusion'] = df['conclusion'].apply(count_veridicality_verbs)


print(df[['argument_id', 'verdicality_count_premise','verdicality_count_conclusion']].describe())

total_verbs_count = sum(verd_verbs_dict.values())
verd_verbs_percent = {}

for verb, count in verd_verbs_dict.items():
    percent = round((count / total_verbs_count) * 100, 2)
    verd_verbs_percent[verb] = percent

print(verd_verbs_percent)

print(signature_count)

print("Most frequent verdicality verb:",max(verd_verbs_percent, key=verd_verbs_percent.get))
print("Least frequent verdicality verb:",min(verd_verbs_percent, key=verd_verbs_percent.get))

print("Most frequent signature_verb:",max(signature_count , key=signature_count.get))
print("Least frequent signature_verb:",min(signature_count , key=signature_count.get))



                +/+           +/-           -/+           ◦/+           ◦/−  \
count  12326.000000  12326.000000  12326.000000  12326.000000  12326.000000   
mean     120.726107    125.177917     16.392666      9.346747     37.228541   
std       68.064036     77.088981     11.251813      5.231935     21.714693   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%       61.000000     57.000000      6.000000      4.000000     18.000000   
50%      118.000000    129.000000     18.000000      9.000000     37.000000   
75%      183.000000    194.000000     26.000000     13.000000     58.000000   
max      235.000000    251.000000     35.000000     19.000000     71.000000   

                −/◦           +/◦           ◦/◦  
count  12326.000000  12326.000000  12326.000000  
mean      14.329872    142.002515   1319.378874  
std        8.774797     84.953227    756.080910  
min        0.000000      0.000000      0.000000  
25%        5.000000     58.000000    6

In [59]:
# load the subset dataset of the collected student essays by Stab and Gurevych
df = pd.read_csv('UKP-InsufficientArguments_v1.0/data-tokenized.tsv', sep='\t', encoding='iso-8859-1')

verd_verbs_dict = {key: 0 for key in veridicality_verbs}

signature_count = {key: 0 for key in signature_list}

def count_veridicality_verbs(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    verb_count = 0
    for token, pos in pos_tags:
        if pos.startswith('V') and stemmer.stem(token.lower()) in stem_words:
            verb  = token.lower()
            if verb in verd_verbs_dict.keys():
                verd_verbs_dict[verb] += 1
            verb_count += 1
    return verb_count

def count_signature_verbs(text):
    for key, values in signature_list.items():
        for value in values:
            if value in text:
                signature_count[key] += 1
    return signature_count

df[['+/+', '+/-', '-/+', '◦/+','◦/−','−/◦','+/◦','◦/◦']] = df['TEXT'].apply(lambda x: pd.Series(count_signature_verbs(x)))

print(df[['TEXT', '+/+', '+/-', '-/+', '◦/+','◦/−','−/◦','+/◦','◦/◦']].describe())


df['veridicality_count'] = df['TEXT'].apply(count_veridicality_verbs)


print(df[['ARGUMENT', 'veridicality_count']].describe())


total_verbs_count = sum(verd_verbs_dict.values())
verd_verbs_percent = {}

for verb, count in verd_verbs_dict.items():
    percent = round((count / total_verbs_count) * 100, 2)
    verd_verbs_percent[verb] = percent

print(verd_verbs_percent)

print(signature_count)

print("Most frequent verdicality verb:",max(verd_verbs_percent, key=verd_verbs_percent.get))
print("Least frequent verdicality verb:",min(verd_verbs_percent, key=verd_verbs_percent.get))

print("Most frequent signature_verb:",max(signature_count , key=signature_count.get))
print("Least frequent signature_verb:",min(signature_count , key=signature_count.get))


               +/+         +/-          -/+     ◦/+          ◦/−          −/◦  \
count  1029.000000  1029.00000  1029.000000  1029.0  1029.000000  1029.000000   
mean     15.186589    11.49174     1.613217     0.0     0.590865     0.862974   
std       8.374281     9.59365     1.134042     0.0     0.491913     1.046431   
min       0.000000     0.00000     0.000000     0.0     0.000000     0.000000   
25%       8.000000     1.00000     1.000000     0.0     0.000000     0.000000   
50%      16.000000    11.00000     1.000000     0.0     1.000000     0.000000   
75%      22.000000    21.00000     3.000000     0.0     1.000000     2.000000   
max      29.000000    28.00000     3.000000     0.0     1.000000     3.000000   

               +/◦          ◦/◦  
count  1029.000000  1029.000000  
mean     25.489796   163.074830  
std      11.739187   107.186268  
min       0.000000     0.000000  
25%      19.000000    64.000000  
50%      27.000000   154.000000  
75%      34.000000   267.000000 